In [1]:
import requests
import io
import pandas as pd
import regex as re

from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text

from unpy_get_dates import *
from scihub_upgraded import SciHub

from datetime import datetime

import warnings
warnings.filterwarnings("ignore","PDFTextExtractionNotAllowedWarning")

## PART 1: Ready Datasets

The complete_one runs are completed for both Scihub & Unpy. We can now merge & analyse the results to mark success/fail at each step of the pipeline, as well as if an artcl and thus a jrnl has_dates.



In [2]:
# SCI-HUB
# NOTE: In Excel, need to replace all None with 'None' for correct splitting

ogi_df = pd.read_csv("20K_Scihub_df.csv", index_col=0, converters={"Scihub_results": lambda x: x.strip("[]").split("', ") if x.startswith('[') else x})
cet_df = pd.read_csv("ceto_df_complete.csv", index_col=0, converters={"Scihub_results": lambda x: x.strip("[]").split("', ") if x.startswith('[') else x})
summ_df = pd.read_csv("summan_df_complete.csv", index_col=0, converters={"Scihub_results": lambda x: x.strip("[]").split("', ") if x.startswith('[') else x})

ogi_df = ogi_df[:20000]

In [3]:
# Scihub Complete One 
sh_comp_one_df = pd.concat([ogi_df, cet_df, summ_df], verify_integrity=True)

In [4]:
# Fixing Error Messages:
 
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,:].Scihub_results.str.replace("article_not_in Scihub", "article_not_in_Scihub")
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,:].Scihub_results.str.replace("cant_open_pdf", "cant_read_pdf")
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==str,:].Scihub_results.str.replace("cant_read_pdf (Zero-Moscow)", "cant_read_pdf", regex=False)

# Removing ' :
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"].map(lambda row: [item.strip("'") for item in row])
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"].map(lambda row: [item.strip('"') for item in row])

# Replacing "None":
sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"] = sh_comp_one_df.loc[sh_comp_one_df.Scihub_results.map(type)==list,"Scihub_results"].map(lambda row: [None if item == "None" else item for item in row])


In [7]:
# sh Total Results:
sh_comp_one_df.Scihub_results.map(type).value_counts() 

<class 'list'>    44343
<class 'str'>      2348
Name: Scihub_results, dtype: int64

In [6]:
# sh_Success:
sh_comp_one_df[sh_comp_one_df.Scihub_results.map(type) == list]["Scihub_results"].map(len).value_counts()

2    22909
3    21434
Name: Scihub_results, dtype: int64

Successful results include:

1- both Received & Accepted Date

2- only Received Date

3- only Accepted Date

In [14]:
# sh_Fail:
sh_comp_one_df[sh_comp_one_df.Scihub_results.map(type) == str]["Scihub_results"].value_counts()

article_not_in_Scihub    2236
cant_read_pdf             111
direct_url_error            1
Name: Scihub_results, dtype: int64

sh_comp_one_df should be ready for analysis

In [84]:
# UNPY (DIRECT URL)
# NOTE: In Excel, need to replace all None with 'None' for correct splitting

unpy_suc_df = pd.read_csv("Unpy_suc_run_v2.csv", index_col=0, converters={"Unpy_results": lambda x: x.replace("None", "'None'"), "Unpy_filter": lambda x: x.strip("[]").split(", ") if x.startswith('[') else x})
unpy_fail_df = pd.read_csv("Unpy_scihub_run.csv", index_col=0, converters={"Scihub_results": lambda x: x.strip("[]").split("', ") if x.startswith('[') else x, "Unpy_filter": lambda x: x.strip("[]").split("', ") if x.startswith('[') else x})

unpy_suc_df["Unpy_results"] = unpy_suc_df.Unpy_results.map(lambda x: x.strip("[]").split("', ") if x.startswith('[') else x)

In [85]:
unpy_fail_df.drop(["OK_link","Rec_date","Acc_date","Unpy_results"], axis=1, inplace=True)
unpy_suc_df.drop(["OK_link","Rec_date","Acc_date"], axis=1, inplace=True)

unpy_fail_df.rename(columns={"Scihub_results":"Unpy_results"}, inplace=True)


In [86]:
# Unpy Complete One 
unpy_comp_one_df = pd.concat([unpy_suc_df, unpy_fail_df], verify_integrity=True)

In [87]:

# Removing ' :
unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"] = unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"].map(lambda row: [item.strip("'") for item in row])
unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"] = unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"].map(lambda row: [item.strip('"') for item in row])

# Replacing "None":
unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"] = unpy_comp_one_df.loc[unpy_comp_one_df.Unpy_results.map(type)==list,"Unpy_results"].map(lambda row: [None if item == "None" else item for item in row])


In [94]:
unpy_comp_one_df.Unpy_results.map(type).value_counts()

<class 'list'>    16171
<class 'str'>      1256
Name: Unpy_results, dtype: int64

In [96]:
unpy_comp_one_df[unpy_comp_one_df.Unpy_results.map(type) == list]["Unpy_results"].map(len).value_counts()

3    9555
2    6616
Name: Unpy_results, dtype: int64

In [105]:
unpy_comp_one_df[unpy_comp_one_df.Unpy_results.map(type) == str]["Unpy_results"].value_counts()

article_not_in Scihub    1211
cant_read_pdf              41
direct_url_error            4
Name: Unpy_results, dtype: int64

Both datasets are ready for analysis!

BUT, need to fix "cant_read_pdf" (?) and "direct_url_error"

In [109]:
from scihub_upgraded import SciHub

In [110]:
sh = SciHub()

In [111]:
# .loc[] & .map() problematic rows for sh_comp_one_df:
sh_comp_one_df.loc[(sh_comp_one_df.Scihub_results == "cant_read_pdf")|(sh_comp_one_df.Scihub_results == "direct_url_error"),"Scihub_results"] = sh_comp_one_df.loc[(sh_comp_one_df.Scihub_results == "cant_read_pdf")|(sh_comp_one_df.Scihub_results == "direct_url_error"),"doi"].map(sh.get_dates)



In [112]:
# sh_Fail:
sh_comp_one_df[sh_comp_one_df.Scihub_results.map(type) == str]["Scihub_results"].value_counts()

article_not_in_Scihub    2236
cant_read_pdf              59
direct_url_error            1
Name: Scihub_results, dtype: int64

In [113]:
# .loc[] & .map() problematic rows for unpy_comp_one_df:
unpy_comp_one_df.loc[(unpy_comp_one_df.Unpy_results == "cant_read_pdf")|(unpy_comp_one_df.Unpy_results == "direct_url_error"),"Unpy_results"] = unpy_comp_one_df.loc[(unpy_comp_one_df.Unpy_results == "cant_read_pdf")|(unpy_comp_one_df.Unpy_results == "direct_url_error"),"doi"].map(sh.get_dates)



In [114]:
unpy_comp_one_df[unpy_comp_one_df.Unpy_results.map(type) == str]["Unpy_results"].value_counts()

article_not_in Scihub    1213
cant_read_pdf              12
direct_url_error            2
Name: Unpy_results, dtype: int64

In [116]:
%store sh_comp_one_df
%store unpy_comp_one_df

Stored 'sh_comp_one_df' (DataFrame)
Stored 'unpy_comp_one_df' (DataFrame)
